<a href="https://colab.research.google.com/github/victwise/DeepLearning/blob/master/Cap%C3%ADtulo8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Implementando la factorización QR

Utilizamos la factorización QR para calcular los valores propios y para calcular la regresión de mínimos cuadrados. Es un bloque de construcción importante en el álgebra lineal numérica.

##Numpy

In [0]:
import numpy as np

np.set_printoptions(suppress=True, precision=4)

In [0]:
n = 5
A = np.random.rand(n,n)
npQ, npR = np.linalg.qr(A)

Comprobando que Q es ortogonal

In [3]:
np.allclose(np.eye(n), npQ @ npQ.T), np.allclose(np.eye(n), npQ.T @ npQ)


(True, True)

Validando que R es una matriz triangular

In [4]:
npR

array([[-1.3832, -1.4192, -0.8742, -0.9388, -0.609 ],
       [ 0.    ,  0.5402,  0.6119,  0.2204,  0.1138],
       [ 0.    ,  0.    , -0.2959, -0.22  , -0.4283],
       [ 0.    ,  0.    ,  0.    , -0.8057, -0.4125],
       [ 0.    ,  0.    ,  0.    ,  0.    , -0.5342]])

##Gram-Schmidt

Classical Gram-Schmidt (inestable)

In [0]:
def cgs(A):
    m, n = A.shape
    Q = np.zeros([m,n], dtype=np.float64)
    R = np.zeros([n,n], dtype=np.float64)
    for j in range(n):
        v = A[:,j]
        for i in range(j):
            R[i,j] = np.dot(Q[:,i], A[:,j])
            v = v - (R[i,j] * Q[:,i])
        R[j,j] = np.linalg.norm(v)
        Q[:, j] = v / R[j,j]
    return Q, R

In [0]:
Q, R = cgs(A)

In [7]:
np.allclose(A, Q @ R)

True

Validar si Q es Unitario

In [8]:
np.allclose(np.eye(len(Q)), Q.dot(Q.T))

True

In [9]:
np.allclose(npQ, -Q)

False

Gram-Schmidt debería recordarle un poco de la iteración de Arnoldi (utilizada para transformar una matriz a la forma de Hessenberg), ya que también es una ortogonalización estructurada.

##Gram-Schmidt modificado

In [0]:
import numpy as np
n = 3
A = np.random.rand(n,n).astype(np.float64)

In [0]:
def mgs(A):
    V = A.copy()
    m, n = A.shape
    Q = np.zeros([m,n], dtype=np.float64)
    R = np.zeros([n,n], dtype=np.float64)
    for i in range(n):
        R[i,i] = np.linalg.norm(V[:,i])
        Q[:,i] = V[:,i] / R[i,i]
        for j in range(i, n):
            R[i,j] = np.dot(Q[:,i],V[:,j])
            V[:,j] = V[:,j] - R[i,j]*Q[:,i]
    return Q, R

In [0]:
Q, R = mgs(A)

In [13]:
np.allclose(np.eye(len(Q)), Q.dot(Q.T.conj()))

True

In [14]:
np.allclose(A, np.matmul(Q,R))

True

##Householder

Las reflexiones de los miembros de la familia conducen a una matriz Q más casi ortogonal con errores de redondeo

Gram-Schmidt se puede detener de manera parcial, dejando un QR reducido de las columnas de la 1ª n de A

In [0]:
import numpy as np
n = 4
A = np.random.rand(n,n).astype(np.float64)

Q = np.zeros([n,n], dtype=np.float64)
R = np.zeros([n,n], dtype=np.float64)

In [17]:
A

array([[0.9592, 0.6772, 0.5466, 0.8434],
       [0.2119, 0.4671, 0.3611, 0.3967],
       [0.1183, 0.8258, 0.5076, 0.6576],
       [0.7123, 0.7728, 0.4586, 0.6199]])

In [0]:
from scipy.linalg import block_diag

In [0]:
np.set_printoptions(5)

##Algoritmo

In [0]:
def householder_lots(A):
    m, n = A.shape
    R = np.copy(A)
    V = []
    Fs = []
    for k in range(n):
        v = np.copy(R[k:,k])
        v = np.reshape(v, (n-k, 1))
        v[0] += np.sign(v[0]) * np.linalg.norm(v)
        v /= np.linalg.norm(v)
        R[k:,k:] = R[k:,k:] - 2*np.matmul(v, np.matmul(v.T, R[k:,k:]))
        V.append(v)
        F = np.eye(n-k) - 2 * np.matmul(v, v.T)/np.matmul(v.T, v)
        Fs.append(F)
    return R, V, Fs